In [7]:
import numpy as np
import pandas as pd


import os
print(os.listdir("input"))
base_add = os.path.join('', 'input')


['HAM10000_images_part_1', 'HAM10000_images_part_1.zip', 'HAM10000_images_part_2', 'HAM10000_images_part_2.zip', 'HAM10000_metadata.csv']


## Importing essential libraries

In [5]:

import matplotlib.pyplot as plt
from glob import glob

from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix



In [ ]:
image_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in glob(os.path.join(base_add, '*', '*.jpg'))}
lesion_type_dict = {'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'}
df = pd.read_csv(os.path.join(base_add, 'HAM10000_metadata.csv'))
print(image_path_dict)

{'ISIC_0024306': 'input\\HAM10000_images_part_1\\ISIC_0024306.jpg', 'ISIC_0024307': 'input\\HAM10000_images_part_1\\ISIC_0024307.jpg', 'ISIC_0024308': 'input\\HAM10000_images_part_1\\ISIC_0024308.jpg', 'ISIC_0024309': 'input\\HAM10000_images_part_1\\ISIC_0024309.jpg', 'ISIC_0024310': 'input\\HAM10000_images_part_1\\ISIC_0024310.jpg', 'ISIC_0024311': 'input\\HAM10000_images_part_1\\ISIC_0024311.jpg', 'ISIC_0024312': 'input\\HAM10000_images_part_1\\ISIC_0024312.jpg', 'ISIC_0024313': 'input\\HAM10000_images_part_1\\ISIC_0024313.jpg', 'ISIC_0024314': 'input\\HAM10000_images_part_1\\ISIC_0024314.jpg', 'ISIC_0024315': 'input\\HAM10000_images_part_1\\ISIC_0024315.jpg', 'ISIC_0024316': 'input\\HAM10000_images_part_1\\ISIC_0024316.jpg', 'ISIC_0024317': 'input\\HAM10000_images_part_1\\ISIC_0024317.jpg', 'ISIC_0024318': 'input\\HAM10000_images_part_1\\ISIC_0024318.jpg', 'ISIC_0024319': 'input\\HAM10000_images_part_1\\ISIC_0024319.jpg', 'ISIC_0024320': 'input\\HAM10000_images_part_1\\ISIC_0024320.

In [39]:
df.sample(10)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10015 entries, 0 to 10014
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   lesion_id     10015 non-null  object 
 1   image_id      10015 non-null  object 
 2   dx            10015 non-null  object 
 3   dx_type       10015 non-null  object 
 4   age           9958 non-null   float64
 5   sex           10015 non-null  object 
 6   localization  10015 non-null  object 
 7   dataset       10015 non-null  object 
dtypes: float64(1), object(7)
memory usage: 626.1+ KB


In [40]:
df['cell_type'] = df['dx'].map(lesion_type_dict.get)
df['path'] = df['image_id'].map(image_path_dict.get)
df['dx_code'] = pd.Categorical(df['dx']).codes
df.sample(3)

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,cell_type,path,dx_code
8873,HAM_0006613,ISIC_0031894,nv,histo,25.0,male,back,rosendahl,Melanocytic nevi,input\HAM10000_images_part_2\ISIC_0031894.jpg,5
2798,HAM_0001457,ISIC_0029856,bcc,histo,75.0,male,back,rosendahl,Basal cell carcinoma,input\HAM10000_images_part_2\ISIC_0029856.jpg,1
7224,HAM_0002750,ISIC_0034136,nv,histo,60.0,female,back,vidir_modern,Melanocytic nevi,input\HAM10000_images_part_2\ISIC_0034136.jpg,5


In [41]:
df['age'].fillna(df['age'].mean(), inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10015 entries, 0 to 10014
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   lesion_id     10015 non-null  object 
 1   image_id      10015 non-null  object 
 2   dx            10015 non-null  object 
 3   dx_type       10015 non-null  object 
 4   age           10015 non-null  float64
 5   sex           10015 non-null  object 
 6   localization  10015 non-null  object 
 7   dataset       10015 non-null  object 
 8   cell_type     10015 non-null  object 
 9   path          10015 non-null  object 
 10  dx_code       10015 non-null  int8   
dtypes: float64(1), int8(1), object(9)
memory usage: 792.3+ KB


In [61]:
df['image'] = df['path'].map(lambda x: np.asarray(Image.open(x).resize((100, 75))))
df

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,cell_type,path,dx_code,image
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,Benign keratosis-like lesions,input\HAM10000_images_part_1\ISIC_0027419.jpg,2,"[[[190, 153, 194], [192, 154, 196], [191, 153,..."
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,Benign keratosis-like lesions,input\HAM10000_images_part_1\ISIC_0025030.jpg,2,"[[[23, 13, 22], [24, 14, 24], [25, 14, 28], [3..."
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,Benign keratosis-like lesions,input\HAM10000_images_part_1\ISIC_0026769.jpg,2,"[[[185, 127, 137], [189, 133, 147], [194, 136,..."
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,Benign keratosis-like lesions,input\HAM10000_images_part_1\ISIC_0025661.jpg,2,"[[[24, 11, 17], [26, 13, 22], [38, 21, 32], [5..."
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,Benign keratosis-like lesions,input\HAM10000_images_part_2\ISIC_0031633.jpg,2,"[[[134, 90, 113], [147, 102, 125], [159, 115, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen,vidir_modern,Actinic keratoses,input\HAM10000_images_part_2\ISIC_0033084.jpg,0,"[[[155, 137, 148], [154, 136, 147], [151, 134,..."
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen,vidir_modern,Actinic keratoses,input\HAM10000_images_part_2\ISIC_0033550.jpg,0,"[[[5, 6, 4], [6, 7, 4], [7, 8, 6], [7, 7, 5], ..."
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen,vidir_modern,Actinic keratoses,input\HAM10000_images_part_2\ISIC_0033536.jpg,0,"[[[112, 102, 105], [127, 116, 117], [141, 127,..."
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face,vidir_modern,Actinic keratoses,input\HAM10000_images_part_2\ISIC_0032854.jpg,0,"[[[158, 122, 146], [159, 122, 141], [154, 112,..."


In [62]:
mask = df.path.str.contains('part_1')
df1 = df[mask]
df2 = df[~mask]

In [15]:
df['image'].map(lambda x: x.shape).value_counts()

(75, 100, 3)    10015
Name: image, dtype: int64

In [16]:
features = df.drop(['dx_code'], axis = 1)
target = df['dx_code']

In [23]:
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size = 0.2)

In [34]:
x_train[0]

array([[[ 1.08396779, -1.02775912, -1.17859676],
        [ 1.06241955, -1.02775912, -1.20014499],
        [ 1.12706425, -0.92001796, -1.07085559],
        ...,
        [ 1.14861249, -0.79072855, -0.72608385],
        [ 1.12706425, -0.79072855, -0.74763208],
        [ 1.10551602, -0.83382502, -0.79072855]],

       [[ 1.10551602, -0.98466266, -1.13550029],
        [ 1.12706425, -0.92001796, -1.02775912],
        [ 1.17016072, -0.87692149, -1.02775912],
        ...,
        [ 1.14861249, -0.72608385, -0.70453562],
        [ 1.14861249, -0.76918032, -0.70453562],
        [ 1.10551602, -0.83382502, -0.81227679]],

       [[ 1.12706425, -0.89846972, -1.04930736],
        [ 1.12706425, -0.89846972, -1.00621089],
        [ 1.21325719, -0.83382502, -0.94156619],
        ...,
        [ 1.14861249, -0.72608385, -0.70453562],
        [ 1.17016072, -0.76918032, -0.76918032],
        [ 1.04087132, -0.87692149, -0.94156619]],

       ...,

       [[ 0.93313015, -0.76918032, -1.04930736],
        [ 0

In [30]:
# Normalization
## maybe not necessary
x_train = np.asarray(x_train['image'].tolist())
x_test = np.asarray(x_test['image'].tolist())

x_train_mean = np.mean(x_train)
x_test_mean = np.mean(x_test)

x_train_std = np.std(x_train)
x_test_std = np.std(x_test)

x_train = (x_train - x_train_mean)/x_train_std
x_test = (x_test - x_test_mean)/x_test_std

In [ ]:
y_train.value_counts()

In [ ]:
# Label Encoding
# y_train = to_categorical(y_train, num_classes = 7)
# y_test = to_categorical(y_test, num_classes = 7)
print(y_train[13])
x_train.shape

In [ ]:
# Reshape images in 3 dimensions
x_train = x_train.reshape(x_train.shape[0], *(75, 100, 3))
x_test = x_test.reshape(x_test.shape[0], *(75, 100, 3))
x_train.shape